In [ ]:
!pip install -q diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 36.6 MB/s eta 0:00:00


In [ ]:
# Imports
import torch
import random
import torchvision
from torch import nn
from PIL import Image
from tqdm.auto import tqdm
from torch.utils.data import Dataset
from matplotlib import pyplot as plt
from torch.nn import functional as F
from torch.utils.data import DataLoader
from diffusers import DDPMScheduler, UNet2DModel
from torchvision import datasets, transforms

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# CONFIG
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

n_epochs = 20

Using device: cuda


# Dataset

In [ ]:
# Define the custom dataset class
class TripletMNISTDataset(Dataset):
    def __init__(self, train=True):
        # Download and prepare the MNIST dataset
        self.mnist_dataset = datasets.MNIST(
            root="./data", train=train, download=True
        )
        self.data = self.mnist_dataset.data
        self.targets = self.mnist_dataset.targets

        # Create a dictionary to store indices of each class
        self.class_indices = {i: torch.where(self.targets == i)[0] for i in range(10)}

        # Transformation to be applied to the images (ToTensor converts to tensor)
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.mnist_dataset)

    def __getitem__(self, index):
        # Anchor sample
        anchor_img = self.data[index].numpy()  # Convert to ndarray
        anchor_label = self.targets[index]

        # Positive sample (same class as anchor)
        positive_index = random.choice(self.class_indices[anchor_label.item()])
        positive_img = self.data[positive_index].numpy()  # Convert to ndarray

        # Negative sample (from a different class)
        negative_label = random.choice([i for i in range(10) if i != anchor_label])
        negative_index = random.choice(self.class_indices[negative_label])
        negative_img = self.data[negative_index].numpy()  # Convert to ndarray

        # Convert to PIL Images
        anchor_img = Image.fromarray(anchor_img, mode='L')
        positive_img = Image.fromarray(positive_img, mode='L')
        negative_img = Image.fromarray(negative_img, mode='L')

        # Apply the transformations (e.g., converting to tensor)
        if self.transform:
            anchor_img = self.transform(anchor_img)
            positive_img = self.transform(positive_img)
            negative_img = self.transform(negative_img)

        return (anchor_img, positive_img, negative_img, anchor_label)

In [ ]:
# Dataset
triplet_dataset = TripletMNISTDataset(train=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 13397733.06it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 401578.30it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3143418.40it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3085605.57it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



# Model

In [ ]:
class ClassConditionedUnet(nn.Module):
    def __init__(self, num_classes=10, class_emb_size=4):
        super().__init__()

        # The embedding layer will map the class label to a vector of size class_emb_size
        self.class_emb = nn.Embedding(num_classes, class_emb_size)

        # Self.model is an unconditional UNet with extra input channels to accept the conditioning information (the class embedding)
        self.model = UNet2DModel(
            sample_size=28,  # the target image resolution
            in_channels=1 + class_emb_size,  # Additional input channels for class cond.
            out_channels=1,  # the number of output channels
            layers_per_block=2,  # how many ResNet layers to use per UNet block
            block_out_channels=(32, 64, 64),
            down_block_types=(
                "DownBlock2D",  # a regular ResNet downsampling block
                "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
                "AttnDownBlock2D",
            ),
            up_block_types=(
                "AttnUpBlock2D",
                "AttnUpBlock2D",  # a ResNet upsampling block with spatial self-attention
                "UpBlock2D",  # a regular ResNet upsampling block
            ),
        )

    # Our forward method now takes the class labels as an additional argument
    def forward(self, x, t, class_labels):
        # Shape of x:
        bs, ch, w, h = x.shape

        # class conditioning in right shape to add as additional input channels
        class_cond = self.class_emb(class_labels)  # Map to embedding dimension
        class_cond = class_cond.view(bs, class_cond.shape[1], 1, 1).expand(bs, class_cond.shape[1], w, h)
        # x is shape (bs, 1, 28, 28) and class_cond is now (bs, 4, 28, 28)

        # Net input is now x and class cond concatenated together along dimension 1
        net_input = torch.cat((x, class_cond), 1)  # (bs, 5, 28, 28)

        # Feed this to the UNet alongside the timestep and return the prediction
        return self.model(net_input, t).sample  # (bs, 1, 28, 28)

# Train

In [ ]:
# Noise Scheduler
noise_scheduler = DDPMScheduler(num_train_timesteps=1000, beta_schedule="squaredcos_cap_v2")

# DataLoader
train_dataloader = DataLoader(triplet_dataset, batch_size=16, shuffle=True)

In [ ]:
# Network
net = ClassConditionedUnet().to(device)

# Optimizer
opt = torch.optim.Adam(net.parameters(), lr=1e-3)
# Losses
loss_fn = nn.MSELoss()
triplet_fn = nn.TripletMarginLoss()

In [ ]:
# Loss Tracking Array
losses = []

# Train
for epoch in range(n_epochs):
  for a, p, n, y in tqdm(train_dataloader):
    a = a.to(device) * 2 - 1  # Data on the GPU (mapped to (-1, 1))
    y = y.to(device)
    noise = torch.randn_like(a)
    timesteps = torch.randint(0, 999, (a.shape[0],)).long().to(device)
    noisy_a = noise_scheduler.add_noise(a, noise, timesteps)
    # Predicted Noise
    pred = net(noisy_a, timesteps, y)
    # MSE Loss
    mse_loss = loss_fn(pred, noise)
    # Triplet Addition
    cnt=0
    triplet_loss = 0
    timesteps_minus_one = timesteps - torch.ones((a.shape[0],)).long().to(device)
    for i in range(0,len(timesteps)):
      if timesteps_minus_one[i]!=-1:
        p = p.to(device) * 2 - 1
        n = n.to(device) * 2 - 1
        noisy_p = noise_scheduler.add_noise(p, noise[i], timesteps_minus_one[i])
        noisy_n = noise_scheduler.add_noise(n, noise[i], timesteps_minus_one[i])
        a_at_timestep_minus_one = noisy_a - pred

        cnt=cnt+1
        triplet_loss += triplet_fn(a_at_timestep_minus_one, noisy_p, noisy_n)

    loss = mse_loss + triplet_loss/cnt if cnt else mse_loss
    # Backpropagation
    opt.zero_grad()
    loss.backward()
    opt.step()

  0%|          | 0/3750 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

...

  0%|          | 0/3750 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

# Evaluate

In [ ]:
# Prepare random x to start from, plus some desired labels y
x = torch.randn(2050, 1, 28, 28).to(device)
y = torch.tensor([[i] * 205 for i in range(10)]).flatten().to(device)

# Sampling loop
for i, t in tqdm(enumerate(noise_scheduler.timesteps)):
    # Get model pred
    with torch.no_grad():
        residual = net(x, t, y)

    # Update sample with step
    x = noise_scheduler.step(residual, t, x).prev_sample

0it [00:00, ?it/s]

In [ ]:
from torchvision.utils import save_image

for i in range(2050):
    class_idx = i%205
    img = x[i].detach().cpu().clip(-1, 1)
    save_image(img, f'/content/drive/MyDrive/Colab Notebooks/triplet_diff/with_triplet/{class_idx}_{i}.png')